In [ ]:
pip install python-terrier==0.10.0 fast-forward-indexes==0.2.0

Start PyTerrier

In [ ]:
import pyterrier as pt

if not pt.started():
    pt.init(
        tqdm="notebook",
        boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"]
    )

Select datasets and index indices

In [ ]:
DATASET_NAME = "beir/fiqa"
TESTSET_NAME = "beir/fiqa/test"
BM25_INDEX_PATH = 'indices/fiqa'
INDEX_PATH = 'indexes/ffindex_fiqa_tct_colbert_msmarco.h5'
FIELDS = ["text"]

SHOULD_RUN_GRID = False
DEVSET_NAME = "irds:beir/fiqa/train"

In [ ]:
import ir_datasets
dataset = pt.get_dataset('irds:' + DATASET_NAME)
ir_ds = ir_datasets.load(DATASET_NAME)

Create or load the sparse index

In [ ]:
from pathlib import Path

idx_path = Path(BM25_INDEX_PATH).absolute()

index_ref = pt.index.IterDictIndexer(
    str(idx_path),
    blocks=True,
    meta={'docno': ir_ds.docs_metadata()['fields']['doc_id']['max_len'] }
).index(dataset.get_corpus_iter(), fields=FIELDS)
index_ref = index_ref.to_memory()

Initialise BM25 and RM3

In [ ]:
from pyterrier.measures import RR, nDCG, MAP

index = pt.IndexFactory.of(str(idx_path))

bm25 = pt.BatchRetrieve(index, wmodel="BM25")
rm3 = pt.rewrite.RM3(index)
testset = pt.get_dataset('irds:' + TESTSET_NAME)

Initialize TCT-ColBERT Encoder

In [ ]:
from fast_forward.encoder import TCTColBERTQueryEncoder, TCTColBERTDocumentEncoder
import torch

q_encoder = TCTColBERTQueryEncoder("castorini/tct_colbert-msmarco")
d_encoder = TCTColBERTDocumentEncoder(
    "castorini/tct_colbert-msmarco",
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)
q_encoder(["Test query 1", "Test query 2"])

Load the FF-index

In [ ]:
from fast_forward import OnDiskIndex, Mode

ff_index = OnDiskIndex.load(
    Path(INDEX_PATH), query_encoder=q_encoder, mode=Mode.MAXP
)
ff_index = ff_index.to_memory()

Create the re-ranking stage for the pipeline

In [ ]:
from fast_forward.util.pyterrier import FFScore
from fast_forward.util.pyterrier import FFInterpolate

ff_score = FFScore(ff_index)
candidates = (bm25 % 5)(testset.get_topics('text')) # Get the candidates
re_ranked = ff_score(candidates)
ff_int = FFInterpolate(alpha=0.05)
ff_int(re_ranked)

Run exhaustive search if required

In [ ]:
if SHOULD_RUN_GRID:
    devset = pt.get_dataset(DEVSET_NAME)
    pt.GridSearch(
        ~bm25 % 100 >> ff_score >> ff_int,
        {ff_int: {"alpha": [0.05, 0.1, 0.5, 0.9]}},
        devset.get_topics(),
        devset.get_qrels(),
        "map",
        verbose=True,
    )
ff_int.alpha

Run the experiment

In [ ]:
result = pt.Experiment(
    [
        bm25,
        bm25 % 5 >> rm3 >> bm25,
        bm25 % 1000 >> ff_score >> ff_int,
        bm25 % 5 >> rm3 >> bm25 % 1000 >> pt.rewrite.reset() >> ff_score >> ff_int,
    ],
    testset.get_topics('text'),
    testset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 10, MAP @ 100],
    names=[
        "BM25",
        "RM3",
        "TCT-ColBERT",
        "RM3+TCT-ColBERT"
    ]
)
result

Save results to CSV files

In [ ]:
import csv   

name_to_csv = {
    "BM25": "results/BM25.csv",
    "RM3": "results/RM3.csv",
    "TCT-ColBERT": "results/BM25_FF.csv",
    "RM3+TCT-ColBERT": "results/BM25_RM3_FF.csv",
}

for index, row in result.iterrows():
    with open(name_to_csv[row['name']], 'a') as f:
        writer = csv.writer(f)
        changed_row = [TESTSET_NAME, row.iloc[1], row.iloc[2], row.iloc[3]]
        writer.writerow(changed_row)